# St. Petersburg Game Alternatives

Due to the counterintuitive nature of the St. Petersburg paradox larger examples are more convincing.  But they also take longer to run.  I want to compare different ways of simulating the game to see what methods are the fastest.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
rng = np.random.default_rng()

In [3]:
player_count = 100_000_000

In [129]:
def pure_python(player_count: int = 10_000_000) -> list[int]:
    res = list()
    for _ in range(player_count):
        payout_round = 1
        while rng.choice(2) == 1:
            payout_round += 1
        res.append(payout_round)
    return res

In [4]:
def pandas_naive(player_count: int = 10_000_000) -> pd.DataFrame:
    players = pd.DataFrame(rng.choice([0,1], size=(1, player_count)))
    while (live_players := players.iloc[-1] == 1).any():
        new_row_index = players.shape[0]
        new_row = rng.choice([0,1], size = (1, player_count))
        players.loc[new_row_index, :] = new_row
        players.loc[new_row_index, ~live_players] = 0
    return players

In [5]:
def pandas_less_rng(player_count: int  = 10_000_000):
    players = pd.DataFrame(rng.choice([0,1], size=(1, player_count)))
    while (live_players := players.iloc[-1] == 1).any():
        new_row_index = players.shape[0]
        players.loc[new_row_index, :] = np.zeros_like(players.iloc[-1])
        players.loc[new_row_index, live_players] = rng.choice([0,1], size=(1, live_players.sum()))
    return players

In [122]:
def pandas_reducing(player_count: int = 10_000_000) -> pd.DataFrame:
    rounds: list[pd.Series] = list()
    rounds.append(pd.Series(rng.choice(2, size=player_count)))
    while not (live_players :=rounds[-1].loc[rounds[-1] == 1].index).empty:
        next_round = pd.Series(rng.choice(2, size=live_players.size), index=live_players)
        rounds.append(next_round)
    game_history = pd.concat(rounds, axis="columns").rename_axis("player").rename_axis("round", axis="columns")
    return game_history



In [130]:
%%timeit

pure_python()

4min 30s ± 3.84 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

pandas_naive()

21.1 s ± 4.25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

pandas_naive(player_count)

4min ± 20.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit

pandas_less_rng()

14.8 s ± 1.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit

pandas_less_rng(player_count)

3min 16s ± 25.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
%%timeit

pandas_reducing()

11.5 s ± 773 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [124]:
%%timeit

pandas_reducing(player_count)

2min 59s ± 8.85 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
